# Import

In [1]:
!pip install torchsummaryX

In [4]:
import torch
import numpy as np
import torch.nn as nn
from torchsummaryX import summary

from sklearn.model_selection import train_test_split


# Constants

In [3]:
num_quantiles = 11 # H_in
look_back_window = 5 # L
hidden_size = 128
n = 50

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load Data

In [41]:
data = np.load('param_training_data.npz', allow_pickle=True)
X = data["X"]
Y = data["Y"]
# Split into train test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=False, test_size=0.1)
print(X.dtype, X_train.dtype)
# X: (N, L, H_in) to (L, N, H_in);
X_train, X_test = np.transpose(X_train, (1, 0, 2)), np.transpose(X_test, (1, 0, 2))
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

float64 float64
(5, 2997, 11) (5, 333, 11) (2997, 11) (333, 11)


In [42]:
X_train[:, :1, :].shape

(5, 1, 11)

# Model

In [43]:
class Network(nn.Module):

    def __init__(self, input_size, output_size, hidden_size): # You can add any extra arguments as you wish

        super(Network, self).__init__()
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, bidirectional=False)

        self.classification = nn.Linear(in_features=hidden_size, out_features=output_size)

    def forward(self, x):

        output, (h_n, c_n) = self.lstm(x)
        # output: (L, 1 * H_out)

        out = self.classification(output)

        return out

model = Network(num_quantiles, num_quantiles, hidden_size).to(device)
summary(model, torch.tensor(X_train[:, :1, :]).to(device)) 

RuntimeError: Expected object of scalar type Double but got scalar type Float for argument #2 'mat2'

In [9]:
x_train = torch.randn(n, look_back_window, 1, num_quantiles) # n
y_train = torch.randn(n, 1, num_quantiles)

# Model Parameters

In [10]:
epochs = 50
lr = 0.001
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Model Training

In [17]:
for epoch in range(epochs):
    arr = np.arange(len(x_train))
    np.random.shuffle(arr)
    for ind in arr:
        seq = x_train[ind]
        labels = y_train[ind]
        optimizer.zero_grad()

        y_pred = model(seq)

        single_loss = loss_function(y_pred[-1, :, :], labels)
        
        single_loss.backward()
        optimizer.step()

    print(f"[LSTM FIT]epoch: {epoch + 1:3} loss: {single_loss.item():10.8f}")

[LSTM FIT]epoch:   1 loss: 0.00138746
[LSTM FIT]epoch:   2 loss: 0.00189571
[LSTM FIT]epoch:   3 loss: 0.00063254
[LSTM FIT]epoch:   4 loss: 0.00513293
[LSTM FIT]epoch:   5 loss: 0.00193271
[LSTM FIT]epoch:   6 loss: 0.00686951
[LSTM FIT]epoch:   7 loss: 0.00086578
[LSTM FIT]epoch:   8 loss: 0.00623362
[LSTM FIT]epoch:   9 loss: 0.00367014
[LSTM FIT]epoch:  10 loss: 0.00237094
[LSTM FIT]epoch:  11 loss: 0.00544881
[LSTM FIT]epoch:  12 loss: 0.00402756
[LSTM FIT]epoch:  13 loss: 0.00496324
[LSTM FIT]epoch:  14 loss: 0.00188385
[LSTM FIT]epoch:  15 loss: 0.00219478
[LSTM FIT]epoch:  16 loss: 0.00066603
[LSTM FIT]epoch:  17 loss: 0.00073235
[LSTM FIT]epoch:  18 loss: 0.00089776
[LSTM FIT]epoch:  19 loss: 0.00103045
[LSTM FIT]epoch:  20 loss: 0.00056660
[LSTM FIT]epoch:  21 loss: 0.00164224
[LSTM FIT]epoch:  22 loss: 0.00133327
[LSTM FIT]epoch:  23 loss: 0.00154545
[LSTM FIT]epoch:  24 loss: 0.00051522
[LSTM FIT]epoch:  25 loss: 0.00065057
[LSTM FIT]epoch:  26 loss: 0.00046732
[LSTM FIT]ep